# Fixed wheat bounding boxes

This notebook fixes some bounding boxes in the training set for the **Global Wheat Detection** dataset. This object detection dataset has almost 3500 wheat head images and their corresponding bounding boxes (~150k bounding boxes).

In [ ]:
import os
import cv2
import csv
import glob
import pandas as pd
import numpy as np
import random
import itertools
from collections import Counter
from math import ceil
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

image_folder_path = "/kaggle/input/global-wheat-detection/train/"

# Reading and Loading the boxes (train csv)

In [ ]:
train = pd.read_csv("/kaggle/input/global-wheat-detection/train.csv")
train

Changing bounding boxes from [x_min, y_min, width, height] to [x_min, y_min, x_max, y_max]

In [ ]:
bboxes = np.stack(train['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x_min', 'y_min', 'width', 'height']):
    train[column] = bboxes[:,i]
    
train["x_max"] = train.apply(lambda col: col.x_min + col.width, axis=1)
train["y_max"] = train.apply(lambda col: col.y_min + col.height, axis = 1)
train.drop(columns=['bbox'], inplace=True)
train.head()

Add column indicating that the image has wheat heads and add .jpg extension to image_id column

In [ ]:
train["class"] = "1"
train["image_id"] = train["image_id"].apply(lambda x: str(x) + ".jpg").astype("str")
train.head()

# Plotting Images with Bounding Boxes

In [ ]:
# all util functions
def draw_rect(img, bboxes, color=(255,0,0)):
    # get an image and return it with all rectangles from bboxes (a 2d-array)
    img = img.copy()
    for bbox in bboxes:
        pt1, pt2 = (bbox[0], bbox[1]), (bbox[2], bbox[3])
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
        img = cv2.rectangle(img.copy(), pt1, pt2, color, int(max(img.shape[:2]) / 500))
    return img

def plot_bboxes(img, bboxes, size=(12, 10)):
    if isinstance(img, str):
        image_file_path = os.path.join(image_folder_path, img)
        img = cv2.imread(image_file_path)[:,:,::-1]

    # get an image and plot it with all bounding boxes
    img2 = draw_rect(img.copy(), bboxes)
    plt.figure(figsize=size)
    plt.imshow(img2)

    
def img_check(image_name_or_index, large=False):
    # find image by image_id or index, plot it with bboxes
    # and returns the image, bounding boxes array and image_id
    if isinstance(image_name_or_index, int):
        unique_images = train.image_id.unique()
        img_file = unique_images[image_name_or_index]
    else:
        img_file = image_name_or_index
    
    bboxes = train.loc[train.image_id == img_file, ['x_min', 'y_min', 'x_max', 'y_max']].values
    image_file_path = os.path.join(image_folder_path, img_file)
    img = cv2.imread(image_file_path)[:,:,::-1]
    print(img_file)
    size = (16, 16) if large else (12, 10)
    plot_bboxes(img, bboxes, size)
    return img, bboxes, img_file


def plot_image(image_id, size=(12, 12)):
    # just find image by id and plot with bounding boxes
    img_file = image_id
    bboxes = train.loc[train.image_id == img_file, ['x_min', 'y_min', 'x_max', 'y_max']].values
    image_file_path = os.path.join(image_folder_path, img_file)
    img = cv2.imread(image_file_path)[:,:,::-1]
    plot_bboxes(img, bboxes, size)
    

def find_bbox_index(image_id, bbox):
    # return the index of the bounding box given the image_id and bounding box coords
    f = train[(train.image_id == image_id) & (train.x_min == bbox[0]) & (train.y_min == bbox[1]) 
              & (train.x_max == bbox[2])  & (train.y_max == bbox[3])]
    return f.index

def plot_bbox_by_number(num):
    plot_bboxes(ximg, xboxes[num:num+1])
    print("Bounding box:", xboxes[num])
    print("Train frame index:", find_bbox_index(ximg_id, xboxes[num]))
    
    
def add_new_bbox(df, image_id, bbox, index=0):
    # return a frame with a new bounding box for the given image_id
    # index=0 will add the new box to last index + 1
    tmp = df[df.image_id == image_id]
    serie = tmp.iloc[0].copy()
    serie[['x_min', 'y_min', 'x_max', 'y_max']] = bbox
    serie['height'] = serie.y_max - serie.y_min
    serie['width'] = serie.x_max - serie.x_min
    serie.name = index if index != 0 else df.index.max() + 1
    return df.append(serie, verify_integrity=True)

## 41c0123cc.jpg

In [ ]:
ximg, xboxes, ximg_id = img_check('41c0123cc.jpg')

In [ ]:
plot_bbox_by_number(6)

In [ ]:
plot_bboxes(ximg, np.array([[0, 15, 140, 80], [290, 270, 370, 420]]))

In [ ]:
# fix image bounding boxes
print(train.shape)
train.drop(173, inplace=True)
train = add_new_bbox(train, ximg_id, [0, 15, 140, 80])
train = add_new_bbox(train, ximg_id, [290, 270, 370, 420])
print(train.shape)
#show fixed image
ximg, xboxes, ximg_id = img_check('41c0123cc.jpg')

## e46378032.jpg

In [ ]:
ximg, xboxes, ximg_id = img_check(6)

In [ ]:
plot_bboxes(ximg, np.array([[240, 0, 300, 80]]))

In [ ]:
# fix image bounding boxes
print(train.shape)
train = add_new_bbox(train, ximg_id, [240, 0, 300, 80])
print(train.shape)
#show fixed image
ximg, xboxes, ximg_id = img_check('e46378032.jpg')

## a22cdd5eb.jpg

In [ ]:
ximg, xboxes, ximg_id = img_check(33)

In [ ]:
train.drop(1021, inplace=True)
ximg, xboxes, ximg_id = img_check('a22cdd5eb.jpg')

## e99cca2a3.jpg

In [ ]:
ximg, xboxes, ximg_id = img_check(43)

In [ ]:
# the first bounding box is totally wrong
# its also missing two bounding boxes
print(xboxes[0])
find_bbox_index(ximg_id, [714, 559, 865, 806])
plot_bboxes(ximg, xboxes[0:1])

In [ ]:
plot_bboxes(ximg, np.array([[730, 560, 810, 710],
                            [810, 715, 860, 798]]))

In [ ]:
train.drop(1274, inplace=True)
train = add_new_bbox(train, ximg_id, [730, 560, 810, 710])
train = add_new_bbox(train, ximg_id, [810, 715, 860, 798])
ximg, xboxes, ximg_id = img_check(ximg_id, large=True)

## a1321ca95.jpg

In [ ]:
ximg, xboxes, ximg_id = img_check('a1321ca95.jpg', large=True)

In [ ]:
plot_bboxes(ximg, xboxes[[13, 23]])
find_bbox_index(ximg_id, xboxes[13])
find_bbox_index(ximg_id, xboxes[23])

In [ ]:
train.drop(2169, inplace=True)
train.drop(2159, inplace=True)
ximg, xboxes, ximg_id = img_check('a1321ca95.jpg', large=True)

In [ ]:
plot_bboxes(ximg, np.array([[300, 450, 510, 620], [140, 610, 400, 820],
                            [95, 520, 350, 615]]), size=(16, 16))

In [ ]:
train = add_new_bbox(train, ximg_id, [300, 450, 510, 620])
train = add_new_bbox(train, ximg_id, [140, 610, 400, 820])
train = add_new_bbox(train, ximg_id, [95, 520, 350, 615])

In [ ]:
plot_image('a1321ca95.jpg')

## 9a30dd802.jpg

In [ ]:
ximg, xboxes, xid = img_check('9a30dd802.jpg')

In [ ]:
plot_bboxes(ximg, xboxes[11:12])
find_bbox_index(xid, [3, 301, 420, 671])

In [ ]:
# fix the bounding box and plot the image
train.loc[52868, 'y_min'] = 450
train.loc[52868, 'x_max'] = 200
train.loc[52868, 'y_max'] = 640
plot_image('9a30dd802.jpg')

## d067ac2b1.jpg

In [ ]:
ximg, xboxes, xid = img_check('d067ac2b1.jpg')

In [ ]:
plot_bboxes(ximg, xboxes[37:39])
find_bbox_index(xid, xboxes[37])
find_bbox_index(xid, xboxes[38]) # resize

In [ ]:
plot_bboxes(ximg, [[620, 5, 900, 135], [580, 130, 720, 206]], size=(16, 16))

In [ ]:
train.loc[121634, 'y_min'] = 300
train.loc[121634, 'x_min'] = 700
train.drop(121633, inplace=True)
train = add_new_bbox(train, 'd067ac2b1.jpg', [620, 5, 900, 135])
train = add_new_bbox(train, 'd067ac2b1.jpg', [580, 130, 720, 206])
plot_image('d067ac2b1.jpg', size=(14, 14))

## 2840176b4.jpg

In [ ]:
imgl = np.unique(train.image_id.values).tolist()
print(imgl[503]); plot_image(imgl[503], size=(12,12))
train.drop(73348, inplace=True) # box at 0, 400

## 284a79f05.jpg

In [ ]:
print(imgl[504]); plot_image(imgl[504], size=(12,12))
train.drop(41429, inplace=True) # box at 0,0

## 29b48c0a7.jpg

In [ ]:
train = add_new_bbox(train, imgl[522], [616, 42, 800, 162])
print(imgl[522]); plot_image(imgl[522], size=(12,12))

## 29f09fa58.jpg

In [ ]:
train = add_new_bbox(train, imgl[525], [978, 930, 1022, 1018])
print(imgl[525]); plot_image(imgl[525], size=(12,12))

# Finding wrong boxes by height

In [ ]:
train.height.hist(bins=30)

In [ ]:
top_height = train[train.height > 300].sort_values(by='height', ascending=False)
xids = top_height.image_id.values
xboxes = top_height[['x_min', 'y_min', 'x_max', 'y_max']].values
top_height

In [ ]:
i = 0; plot_bboxes(xids[i], xboxes[i:i+1], size=(10,8))

In [ ]:
i = 1; plot_bboxes(xids[i], xboxes[i:i+1], size=(10,8))

In [ ]:
# remove wrong bounding boxes
for i in [0, 1]:
    idx = find_bbox_index(xids[i], xboxes[i])
    train.drop(idx, inplace=True)

# Finding wrong boxes by width

In [ ]:
top_width = train[train.width > 300].sort_values(by='width', ascending=False)
xids = top_width.image_id.values
xboxes = top_width[['x_min', 'y_min', 'x_max', 'y_max']].values
top_width

In [ ]:
i = 0; plot_bboxes(xids[i], xboxes[i:i+1], size=(10,8))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[3687, 'x_max'] = 132
train.loc[3687, 'y_max'] = 215
plot_image(xids[0])

In [ ]:
i = 7; plot_bboxes(xids[i], xboxes[i:i+1], size=(14,12))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[147552, 'x_min'] = 408
plot_image(xids[7], size=(14, 14))

In [ ]:
i = 14; plot_bboxes(xids[i], xboxes[i:i+1], size=(14,12))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[4412, 'x_min'] = 538
plot_image(xids[i], size=(14, 14))

In [ ]:
i = 32; plot_bboxes(xids[i], xboxes[i:i+1], size=(14,12))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[86917, 'y_max'] = 455
train.drop(86913, inplace=True)
train.drop(86922, inplace=True)
train = add_new_bbox(train, 'd561f23d2.jpg', [376, 422, 488, 490])
plot_image('d561f23d2.jpg')

# Filter boxes by median size ratio

In [ ]:
train.height = train.y_max - train.y_min
train.width = train.x_max - train.x_min
train['bbox_size'] = train.height * train.width
train['median_bbox_size'] = train.groupby('image_id')['bbox_size'].transform('median')
train['median_ratio_diff'] = (train.bbox_size - train.median_bbox_size) / train.median_bbox_size

sort_train = train.sort_values(by='median_ratio_diff', ascending=False)
xboxes = sort_train[['x_min', 'y_min', 'x_max', 'y_max']].values
xids = sort_train['image_id'].values
sort_train

In [ ]:
i=0
plot_image(xids[i], size=(12, 12))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[117344, 'x_min'] = 468
train.loc[117344, 'y_max'] = 356
plot_image(xids[i], size=(10, 10))

In [ ]:
i=1
plot_image(xids[i], size=(12, 12))
find_bbox_index(xids[i], xboxes[i])

In [ ]:
train.loc[147504, 'x_max'] = 420
train.loc[147504, 'y_max'] = 155
plot_image(xids[i], size=(16, 16))

In [ ]:
# two bounding boxes without visible wheat heads
i=-1
plot_image(xids[i], size=(14, 14))
find_bbox_index(xids[i], [483,  31, 534,  89])
find_bbox_index(xids[i], [563,  41, 608,  85])

In [ ]:
train.drop(126792, inplace=True)
train.drop(126793, inplace=True)
plot_image(xids[i], size=(14, 14))

# Filter very small bounding boxes

In [ ]:
train['smallest_dim'] = train.apply(lambda x: min(x['width'], x['height']), axis=1)
sort_train = train.sort_values(by='smallest_dim', ascending=True)
sort_train.head()
xboxes = sort_train[['x_min', 'y_min', 'x_max', 'y_max']].values
xids = sort_train['image_id'].values

In [ ]:
# Example of "click-error" box (top right corner)
i = 80
print(sort_train.iloc[i])
plot_bboxes(xids[i], [xboxes[i]], size=(14, 14))

In [ ]:
# Filter every bounding box where the height or width is less than 9px
train = train[train.smallest_dim >= 9]
train.shape

# Save new train csv file

In [ ]:
# recalculate the height and width of bounding boxes
train.height = train.y_max - train.y_min
train.width = train.x_max - train.x_min
# remove dummy columns
train.drop(['bbox_size', 'median_bbox_size', 'median_ratio_diff', 'smallest_dim'], axis=1, inplace=True)
# save to new csv file (note: we keep the same original index)
train.to_csv('gwd_train_fixed', index=False)
train